# Computing Clusters' $R_{\rm vir}$'s

## 1. Preamble

In [1]:
from __future__ import division

import math

import numpy as np
from numpy.linalg import norm
from numpy import pi, sqrt, log, log10, exp, power

from scipy.optimize import fsolve

import os

In [7]:
%config InlineBackend.figure_format = "retina"
matplotlib.rcParams['figure.figsize'] = (13,8)

In [2]:
data_path = './likelihoods/'

## 2. Useful Constants and Functions

In [3]:
c0 = 299792458. # [m/s] speed of light
Mpc_over_m = 3.085677581282e22 # Mpc/m conversion
Mpc_over_cm = Mpc_over_m*100. # Mpc/cm conversion
kpc_over_cm = Mpc_over_cm/1000. # kpc/cm conversion

_rads_over_arcsec_ = (2.*pi)/(360.*60.*60.) # [rad/arcsec]

hbarc = 197.32698045930252e-16 # [GeV*cm]
GeV_times_cm = 1./hbarc # GeV*cm conversion
Mpc_times_GeV = GeV_times_cm * Mpc_over_cm # Mpc*GeV conversion

GeV_over_g = 1.78266192e-24

In [4]:
def rho_bg(h, OmL, z):
    """
    The background density [g/cm^3]
    """
    
    rho_crit_GeV4 = 8.0959*(h**2)*(10**-47)
    rho_crit_gcm3 = rho_crit_GeV4 * (GeV_over_g * GeV_times_cm**3.)
    
    return rho_crit_gcm3*(OmL + (1.-OmL)*(1.+z)**3.)



def nfw(rho0, rs, r):
    """
    NFW density [g/cm^3]
    """
    return rho0/((r/rs) * (1 + r/rs)**2)



def mass(rho0, rs, r):
    """
    DM cluster halo mass [g]
    """
    
    rs_cm = rs*kpc_over_cm
    
    return 4*pi*(rs_cm**3.)*rho0*(log(1+r/rs) + 1./(1+r/rs))



def avg_dens(rho0, rs, r):
    """
    Average DM halo density [g/cm^3]
    """
    
    r_cm = r*kpc_over_cm
    
    return mass(rho0, rs, r)/(4.*pi/3. * r_cm**3.)



def Rvir(rho0, rs, rho_compare):
    """
    Virial radius [kpc]
    """
    
    dens = lambda rr: avg_dens(rho0, rs, rr) - rho_compare
    
    return fsolve(dens, rs)

## 3. Reading Data

In [6]:
names = []
z_cls = np.array([])

DA_cls = np.array([])

ne0_cls = np.array([])
beta_cls = np.array([])
rc_out_cls = np.array([])
f_cls = np.array([])
rc_in_cls = np.array([])

Ndm_cls = np.array([])
rs_cls = np.array([])

with open(data_path+'bonamente_add.txt', 'r') as filein:
  for i, line in enumerate(filein):
    if line.strip() and line.find('#') == -1:
      this_line = line.split()
      
      names.append(this_line[0]+' '+this_line[1])
      z_cls = np.append(z_cls, float(this_line[2]))
      
      DA_cls = np.append(DA_cls, float(this_line[3]))
      
      ne0_cls = np.append(ne0_cls, float(this_line[6]))
      beta_cls = np.append(beta_cls, float(this_line[8]))
      rc_out_cls = np.append(rc_out_cls, float(this_line[10]))
      f_cls = np.append(f_cls, float(this_line[12]))
      rc_in_cls = np.append(rc_in_cls, float(this_line[14]))
      Ndm_cls = np.append(Ndm_cls, float(this_line[16]))
      rs_cls = np.append(rs_cls, float(this_line[18]))

rc_out_cls = (DA_cls*1.e3)*(_rads_over_arcsec_*rc_out_cls) # converting from arcsec to kpc
rc_in_cls = (DA_cls*1.e3)*(_rads_over_arcsec_*rc_in_cls) # converting from arcsec to kpc

rs_cls = (DA_cls*1.e3)*_rads_over_arcsec_ * rs_cls
Ndm_cls *= 1.e-25

In [8]:
print Ndm_cls
print rs_cls

[4.700e-26 9.200e-26 6.500e-26 1.780e-25 5.790e-25 1.020e-25 1.800e-26
 2.680e-25 2.600e-26 1.220e-25 1.360e-25 4.700e-26 2.020e-25 3.040e-25
 2.800e-26 3.290e-25 7.600e-26 1.730e-25 6.600e-26 7.000e-27 5.800e-26
 1.630e-25 4.100e-26 4.570e-25 6.600e-26 7.590e-25 1.000e-26 7.400e-26
 1.830e-25 2.700e-26 6.050e-25 2.600e-26 1.201e-24 2.700e-26 1.730e-25
 3.400e-26 4.000e-27 4.090e-25]
[ 457.56715223  354.88361457  396.48062841  257.14517646  172.78759595
  351.97473249 1087.92190041  236.34666954  983.20214529  256.56340004
  240.6615113   533.68290017  218.1661565   187.33200638  778.12595818
  213.80283337  396.77151662  242.01898961  486.89837994 2071.17252707
  493.05551369  267.03537556  597.38741786  218.74793292  446.80428851
  127.11814719 1505.34647985  426.63603938  238.382887    757.27896989
  128.96043918  480.93517166  134.3903524   765.61776521  221.07503859
  736.1410934  4294.38262453  240.85543678]


In [21]:
h_early = 0.69
h_late = 0.74
h_mean = (h_early + h_late)/2.
OmL = 0.69
bigger = 200. # the virial radius is defined where the cluster mean density is 200 times bigger than the background density

Rvirs_cls = np.array([])

for i, name in enumerate(names):
    rvir = Rvir(Ndm_cls[i], rs_cls[i], bigger*rho_bg(h_mean, OmL, z_cls[i]))[0]
    rvir_arcsec = rvir/(DA_cls[i]*1.e3 *_rads_over_arcsec_)
    
    Rvirs_cls = np.append(Rvirs_cls, rvir_arcsec)

In [22]:
Rvirs_cls * (DA_cls*1.e3 *_rads_over_arcsec_)

array([2286.76085318, 2276.68843426, 2220.10862363, 2099.32134941,
       2194.50233947, 2325.05753711, 3722.69478112, 2239.25636633,
       3833.86906984, 1782.87451295, 1737.19716603, 2573.85439214,
       1823.12720518, 1821.36927729, 3055.68402186, 2123.36309677,
       2243.58064706, 1861.58118168, 2600.18884131, 4685.5255173 ,
       2468.626059  , 1933.59582719, 2526.36045527, 2255.62437574,
       2190.73463917, 1557.13210109, 3530.34128152, 2124.34897171,
       1670.03450509, 2569.13374254, 1399.78501493, 1584.45241363,
       1872.32257612, 2440.55365196, 1359.13158994, 2415.47824557,
       6310.58915843, 1951.97774231])

In [23]:
with open(data_path+'bonamente_add.txt', 'r') as filein:
    original_lines = filein.readlines()

print original_lines[2]
print len(lines[4:])
print original_lines[4]

original_lines[2] = original_lines[2].rstrip('\n')
original_lines[2] += '   R_vir [arcsec; {}xbackground w/ h={}; Omega_Lambda={}]\n'.format(bigger, h_mean, OmL)

print original_lines[2]

data = original_lines[4:]
new_data = []

for i, line in enumerate(data):
    line = line.rstrip('\n')
    line += '    {:2}\n'.format(round(Rvirs_cls[i],2))
    
    new_data.append(line)

new_lines = [ pre_line for pre_line in original_lines[:4] ]

for line in new_data:
    new_lines.append(line)

with open(data_path+'add.txt', 'w') as file_out:
    file_out.writelines(new_lines)

# cluster name        z   DA [Mpc]  +err    -err    ne0 [cm^-3]     ne0_err     beta    beta_err     rc_out [arcsec]      rc_out_err     f   f_err   rc_in [arcsec]   rc_in_err     N [1.e-25 g cm^-3]      N_err   rs  [arcsec]    rs_err

38
Abell 1413          0.142   780.    180.    -130.   3.66e-2     0.535e-2    0.531   0.016   39.3    4.1    0.76     0.02   6.5  1.4     0.47    0.425   121.     49.

# cluster name        z   DA [Mpc]  +err    -err    ne0 [cm^-3]     ne0_err     beta    beta_err     rc_out [arcsec]      rc_out_err     f   f_err   rc_in [arcsec]   rc_in_err     N [1.e-25 g cm^-3]      N_err   rs  [arcsec]    rs_err   R_vir [arcsec; 200.0xbackground w/ h=0.715; Omega_Lambda=0.69]



In [24]:
R_vir_read_cls = np.array([])

with open(data_path+'add.txt', 'r') as filein:
  for i, line in enumerate(filein):
    if line.strip() and line.find('#') == -1:
      this_line = line.split()
      
      R_vir_read_cls = np.append(R_vir_read_cls, float(this_line[20]))

In [25]:
R_vir_read_cls/Rvirs_cls

array([1.00000703, 1.00000362, 0.99999372, 0.99999605, 1.00000309,
       0.99999637, 1.00000375, 0.99999398, 0.99999714, 1.000002  ,
       0.99999404, 0.99999477, 0.99999208, 1.00000591, 1.00000544,
       1.00000211, 1.00000578, 0.99999927, 1.00000364, 0.99999947,
       0.9999973 , 1.00000108, 0.99999875, 0.99999672, 1.0000012 ,
       0.99999891, 0.99999923, 0.99999446, 1.0000109 , 0.99998816,
       1.00001148, 0.99999275, 0.99999805, 0.99998982, 1.00004811,
       1.00000933, 1.00000403, 0.99999923])